In [17]:
import os
import numpy as np
import cv2
import keras
from keras.models import Sequential
from keras.layers import *
from keras import backend as k
from keras import optimizers
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.engine.topology import Layer
from PIL import Image
from glob import glob

In [2]:
BATCH_SIZE = 32
batch_test = 4
root_dir = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
data_directory = os.path.join(root_dir, 'lfw-deepfunneled') # Make a directory to store the data and enter it here.
                    # We will be using a smaller dataset (LFW) than the one used in the paper (CelebA) for computational resource considerations.
                    # Download it from http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz.
resize_dir = os.path.join(root_dir, 'Resized_images')
#device = -1
#lambda_ = {'feature': 1., 'pixel': 1., 'total_variation': 1e-5}
model_directory = os.path.join(root_dir, 'NIPS_model_ass1') # Make a directory to store the models and enter it here. Move Vgg4Layers.npz to the model directory.
outsize = (96, 96)

In [ ]:
# Resizing and saving each portrait in the data directory

'''iterator = 0

for root, dirs, files in os.walk(data_directory):
    for fname in files:

        filepath = os.path.join(root, fname)
        
        img = cv2.imread(filepath)
        
        img1 = cv2.resize(img, (96, 96))
        
        cv2.imwrite(os.path.join(resize_dir, '%r_1.jpg' %iterator), img1)
        
        iterator += 1'''

In [3]:
def render(img_rgb):
    """
    Applies pencil sketch effect to an RGB image
    :param img_rgb: RGB image to be processed
    :returns: Processed RGB image
    """
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (21, 21), 0, 0)
    img_blend = cv2.divide(img_gray, img_blur, scale=256)
    
    return img_blend

In [4]:
def img_load(img):
    
    t_ = []
    x_ = []
    for i in img:
        t = np.asarray(Image.open(i).convert('RGB').resize((96, 96), Image.LANCZOS), 'f').transpose(2, 0, 1)
        x = render(np.asarray(Image.open(i).convert('RGB').resize((96, 96), Image.LANCZOS), 'f'))[None]
        
        t_.append(t)
        x_.append(x)
    
    return np.asarray(t_), np.asarray(x_)

In [5]:
data_file = sorted(glob('{}/*.jpg'.format(resize_dir)))
training_set = img_load(data_file[:int(.64 * len(data_file))])
validation_set = img_load(data_file[int(.64 * len(data_file)) : int(.8 * len(data_file))])
test_set = img_load(data_file[int(.8 * len(data_file) ):])

batches = int(len(training_set)/BATCH_SIZE)

In [16]:
def batch_generator():
    
    while True:

        for batch in range(batches):
            
            data = training_set[1][batch * BATCH_SIZE: (batch + 1) * BATCH_SIZE]
            target = training_set[0][batch * BATCH_SIZE: (batch + 1) * BATCH_SIZE]
                
            yield (data, target)

In [18]:
def residual_block(y):
    shortcut = y
    
    y = Conv2D(128, 3, 1, padding = 'same')(y)
    y = BatchNormalization(axis = 0, scale = None)(y)
    y = Activation('relu')
    
    y = Conv2D(128, 3, 1, padding = 'same')(y)
    y = BatchNormalization(axis = 0, scale = None)(y)
    y = Activation('relu')

    y = layers.add([shortcut, y])

    return y

In [ ]:
model = Sequential()

model.add(ZeroPadding2D(4, input_shape = (1, 96, 96), data_format='channels_first'))

model.add(Conv2D(32, 9, 1))

model.add(BatchNormalization(axis = 0, scale = None))

model.add(Activation('relu'))

model.add(Conv2D(64, 3, 2, padding = 'same'))

model.add(BatchNormalization(axis = -1, scale = None))

model.add(Activation('relu'))

model.add(Conv2D(128, 3, 2, padding = 'same'))

model.add(BatchNormalization(axis = 0, scale = None))

model.add(Activation('relu'))

input1 = Input(shape = (1, 96, 96))
model1 = model(input1)

res1 = residual_block(model1)

res2 = residual_block(res1)

res3 = residual_block(res2)

res4 = residual_block(res3)

res5 = residual_block(res4)